In [1]:
# notebooks/06_Analisis_de_Sentimientos.ipynb
import os
import logging
import sys
import pandas as pd
# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from utils.file_utils import read_excel_file, save_data_to_excel
from config import XLSX_DIRECTORY

def analyze_sentiment_vader(sentence):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    sentiment_score = sentiment_analyzer.polarity_scores(sentence)
    return sentiment_score['compound']  # Devuelve la puntuación compuesta

def categorize_sentiment(score):
    if -1.0 <= score <= -0.6:
        return 'Muy Negativo'
    elif -0.6 < score <= -0.2:
        return 'Negativo'
    elif -0.2 < score <= 0.2:
        return 'Neutral'
    elif 0.2 < score <= 0.6:
        return 'Positivo'
    elif 0.6 < score <= 1.0:
        return 'Muy Positivo'
    else:
        return 'Indefinido'


def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel para análisis de sentimientos...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

    # Preparar DataFrame para los resultados del análisis de sentimientos
    sentiment_df = pd.DataFrame(columns=["id_sentiment", "id_sentence",  "sentiment_score", "sentiment_category"])

    # Realizar análisis de sentimientos
    logging.info("Realizando análisis de sentimientos...")
    for index, row in sentence_data.iterrows():
        sentence = str(row["sentence_clean"])  # Convertir a cadena de texto
        sentiment_score = analyze_sentiment_vader(sentence)
        sentiment_category = categorize_sentiment(sentiment_score)
        sentiment_df.loc[index] = [index + 1, row["id_sentence"], sentiment_score, sentiment_category]  # index + 1 para iniciar en 1

    # Guardar los resultados del análisis de sentimientos en un nuevo archivo Excel
    logging.info("Guardando resultados del análisis de sentimientos en Excel...")
    save_data_to_excel(sentiment_df, XLSX_DIRECTORY, "sentiment.xlsx")

if __name__ == "__main__":
    main()

In [ ]:
#pip install vaderSentiment


In [1]:
import os
import logging
import sys
import pandas as pd
import torch
# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from utils.file_utils import read_excel_file, save_data_to_excel
from config import XLSX_DIRECTORY
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Carga de modelos y tokenizers para el análisis de sentimiento
# Define el nombre del modelo y carga el tokenizer y el modelo
beto_model_name = "finiteautomata/beto-sentiment-analysis"
beto_tokenizer = AutoTokenizer.from_pretrained(beto_model_name)
beto_model = AutoModelForSequenceClassification.from_pretrained(beto_model_name)


# Cargar los datos desde el archivo Excel
logging.info("Cargando datos de oraciones desde Excel para análisis de sentimientos...")
df = pd.read_excel(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

def analyze_sentiment(sentence, tokenizer, model):
    if not isinstance(sentence, str):
        logging.warning(f"Se encontró un valor no textual: {sentence}")
        # Devolver None para el score y la categoría si no es un texto
        return None, None
    inputs = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.squeeze()
    probs = softmax(logits, dim=0)
    sentiment_score = round(probs.max().item(), 4)
    sentiment_category = model.config.id2label[probs.argmax().item()]
    
    # Imprimir los logits y probabilidades para depuración
    #print(f"Logits: {logits}")
    #print(f"Probabilidades: {probs}")
    #print(f"Categoría de Sentimiento: {sentiment_category}")
    
    return sentiment_score, sentiment_category


def main():

    # Cargar los datos desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel para análisis de sentimientos...")
    sentence_data = pd.read_excel(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

    # Preparar DataFrame para los resultados del análisis de sentimientos
    sentiment_df = pd.DataFrame(columns=["id_sentiment", "id_sentence", "sentiment_score", "sentiment_category"])

    # Realizar análisis de sentimientos
    logging.info("Realizando análisis de sentimientos...")
    for index, row in sentence_data.iterrows():
        sentiment_score, sentiment_category = analyze_sentiment(row["sentence_clean"], beto_tokenizer, beto_model)
        # sentiment_df.loc[index] = [index + 1, row["id_sentence"], sentiment_score, sentiment_category]
        # Asegurarse de que el valor no es None antes de asignarlo
        if sentiment_score is not None and sentiment_category is not None:
            sentiment_df.loc[index] = [index + 1, row["id_sentence"], sentiment_score, sentiment_category]



    # Guardar los resultados del análisis de sentimientos en un nuevo archivo Excel
    logging.info("Guardando resultados del análisis de sentimientos en Excel...")
    sentiment_df.to_excel(os.path.join(XLSX_DIRECTORY, "sentiment.xlsx"), index=False)

if __name__ == "__main__":
    main()


In [6]:
import os
import logging
import sys
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from utils.file_utils import read_excel_file, save_data_to_excel
from config import XLSX_DIRECTORY
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Carga de modelos y tokenizers para el análisis de sentimiento
beto_model_name = "finiteautomata/beto-sentiment-analysis"
beto_tokenizer = AutoTokenizer.from_pretrained(beto_model_name)
beto_model = AutoModelForSequenceClassification.from_pretrained(beto_model_name)

# Función para analizar el sentimiento del contexto de cada entidad
def analyze_sentiment(context, tokenizer, model):
    # Verificar si el contexto es una cadena de texto válida
    if not isinstance(context, str):
        logging.warning(f"Se encontró un valor no textual: {context}")
        # Devolver None para el score y la categoría si no es un texto
        return None, None

    # Continuar con el análisis de sentimientos si el contexto es una cadena de texto
    inputs = tokenizer(context, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.squeeze()
    probs = softmax(logits, dim=0)
    sentiment_score = round(probs.max().item(), 4)
    sentiment_category = model.config.id2label[probs.argmax().item()]
    return sentiment_score, sentiment_category


def main():
    # Cargar los datos desde el archivo Excel
    logging.info("Cargando datos desde Excel para análisis de sentimientos a nivel de entidad...")
    entity_data = pd.read_excel(os.path.join(XLSX_DIRECTORY, "resultado_sentimiento.xlsx"))

    # Realizar análisis de sentimientos a nivel de entidad
    logging.info("Realizando análisis de sentimientos a nivel de entidad...")
    for index, row in entity_data.iterrows():
        sentiment_score, sentiment_category = analyze_sentiment(row["context"], beto_tokenizer, beto_model)
        # Asegurarse de que el valor no es None antes de asignarlo
        if sentiment_score is not None and sentiment_category is not None:
            entity_data.at[index, "sentiment_score"] = sentiment_score
            entity_data.at[index, "sentiment_category"] = sentiment_category
        else:
            entity_data.at[index, "sentiment_score"] = None
            entity_data.at[index, "sentiment_category"] = None

    # Guardar los resultados del análisis de sentimientos en un nuevo archivo Excel
    output_file = os.path.join(XLSX_DIRECTORY, "sentimientos_analisis_2.xlsx")
    logging.info(f"Guardando resultados en {output_file}...")
    entity_data.to_excel(output_file, index=False)

if __name__ == "__main__":
    main()


: 